# Pydantic basics
* work with data in an OOP manner
* fields and methods
* validate data
* serialize - OOP object -> json
* deserialize - json -> OOP object

### start without pydantic
how to naively create validation for a Person class

- error handling
- property
   - getter
   - setter (put in validation here)

In [1]:
class Person: 
    def __init__(self, name, gender, age):
        self.name = name
        self.gender = gender
        self.age = age


p1 = Person("Gauss", "M", 73)

# default __repr__ for the Person class
p1
        

In [2]:
class Person:
    def __init__(self, name , gender, age):
        self.name = name
        self.gender = gender
        self.age = age

    def __repr__(self):
        return f"Person(name='{self.name}', gender='{self.gender}', age={self.age})"


p1 = Person("Gauss", "M", 73)

p1

Person(name='Gauss', gender='M', age=73)

In [3]:
p2 = Person('Dadda', 'F', 3)
p2

Person(name='Dadda', gender='F', age=3)

In [4]:
p1.age , p1.name 

(73, 'Gauss')

In [5]:
p2.age , p2.name 

(3, 'Dadda')

create a totally invalid person

In [6]:
# this should give error
Person(3.141592, 2.715, 'TwentyFive')

Person(name='3.141592', gender='2.715', age=TwentyFive)

some other programming languages have static typing or hard typing

in hard typing language

```

int age;
string name;
string gender;

```

it will give error when giving wrong data types to the variable

```
age = "femtiotvå"; // -> gives an error
```


in python we have dynamically typed language lets try type hinting

In [7]:
class Person:
    def __init__(self, name: str, gender: str, age: int):
        self.name = name
        self.gender = gender
        self.age = age

    def __repr__(self):
        return f"Person(name='{self.name}', gender='{self.gender}', age={self.age})"

Person(3.14, "M", 53)

Person(name='3.14', gender='M', age=53)

### add property

In [8]:
class Person:
    def __init__(self, name: str, gender: str, age: int):
        self.name = name
        self.gender = gender
        self.age = age

    @property
    def age(self):
        print("getter called")
        return self._age
    
    @age.setter
    def age(self, age):
        print("setter called, validate age code here")
        self._age = age

    def __repr__(self):
        return f"Person(name='{self.name}', gender='{self.gender}', age={self.age})"
    
p3 = Person('henry', gender = 'M', age = 32)
p3

setter called, validate age code here
getter called


Person(name='henry', gender='M', age=32)

In [9]:
p3.age 

getter called


32

### validation on property

In [ ]:
class Person:
    def __init__(self, name: str, gender: str, age: int):
        self.name = name
        self.gender = gender
        self.age = age

    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, age: int):

        if not isinstance(age, int):
            raise TypeError(f"age must be an int, not type {type(age)}") 

        if not 0<=age<=125:
            raise ValueError(f"age must be between 0 and 125, not {age}") 
        
        self._age = age

    def __repr__(self):
        return f"Person(name='{self.name}', gender='{self.gender}', age={self.age})"

try:
    p4 = Person("henry", gender = "M", age = "fem")
except TypeError as err:
    print(err)

try:
    Person("henry", gender = "M", age = -539128)
except ValueError as err:
    print(err)

age must be an int, not type <class 'str'>
age must be between 0 and 125, not -539128
